# Code to Scrape 'Missed games due to injury'

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
import os

#-----------------------------------------Defind Function ----------------------------------------
# Function to scrape NBA injury data
def scrape_nba_injury_data(start_date, end_date):
    all_data = []  # List to store data from all pages

    # Convert start_date and end_date strings to datetime objects
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.strptime(end_date, '%Y-%m-%d')

    current_date = start_date
    while current_date <= end_date:
        url = f" https://prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate={current_date.strftime('%Y-%m-%d')}&EndDate={current_date.strftime('%Y-%m-%d')}&InjuriesChkBx=yes&Submit=Search"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extracting table data
        table = soup.find('table', {'class': 'datatable'})
        if table is not None:
            rows = table.find_all('tr')
            for row in rows[1:]:  # Skip the header row
                cols = row.find_all('td')
                # Remove bullet points from Acquired and Relinquished columns
                cols = [col.text.strip().replace('•', '') for col in cols]
                all_data.append(cols)
        
        current_date += timedelta(days=1)
#-----------------------------------------Scrape Data----------------------------------------        
    # Converting to DataFrame
    df = pd.DataFrame(all_data, columns=['Date', 'Team', 'Acquired', 'Relinquished', 'Notes'])
    return df
injury_data = scrape_nba_injury_data('2013-10-29', '2023-06-12')
if not injury_data.empty:
    print("NBA Injury Data:")
    print(injury_data)
    #-----------------------------------------Save Data----------------------------------------
    # define folder path
    savepath_csv = '/Users/nya/NBA project/NBA Project/00/scraped datasets'
    # Save DataFrame to CSV file
    file_path = os.path.join(savepath_csv, 'nba_injury_data.csv')
    injury_data.to_csv(file_path, index=False)
    print(f"DataFrame saved as CSV file in {file_path}.")

NBA Injury Data:
            Date       Team         Acquired                    Relinquished  \
0     2013-10-29      Bucks                                   Carlos Delfino   
1     2013-10-29  Cavaliers                                    Carrick Felix   
2     2013-10-29    Celtics                                      Rajon Rondo   
3     2013-10-29      Hawks                    Louis Williams / Lou Williams   
4     2013-10-29      Magic                                       Doron Lamb   
...          ...        ...              ...                             ...   
9673  2023-04-30     Knicks   Quentin Grimes                                   
9674  2023-05-02     Knicks    Julius Randle                                   
9675  2023-05-02       Suns                                       Chris Paul   
9676  2023-05-10     Knicks                                    Evan Fournier   
9677  2023-05-11       Suns                                    Deandre Ayton   

                      